# Original Data calling

In [1]:
import pandas as pd

raw_2D =  pd.read_csv('final_sep_dar.txt',delimiter="$")

print raw_2D.shape,raw_2D.size

raw_2D.head()

(13085, 9) 117765


,class,sample,date,Peak,RT,scan_num,intensity,compoud,Qual
0,Y,Y7E,2015/11/17,1,2.142,549,37101,Carbon dioxide (CAS),4
1,Y,Y7E,2015/11/17,2,2.239,575,200880,Carbon dioxide (CAS),4
2,Y,Y7E,2015/11/17,3,7.990,2109,3315846,Ethanol (CAS),64
3,Y,Y7E,2015/11/17,4,8.076,2132,483523,2,90
4,Y,Y7E,2015/11/17,5,8.188,2162,67817,3-Cyclopropyl-1-butyne,9


# unit Scan number to time

In [2]:
ten_sec = 1.0/60 * 10
print "10 second as minite : ",ten_sec

ten_sec2scan = ten_sec/(raw_2D.RT/raw_2D.scan_num)
ten_sec2scan_avg = ten_sec2scan.mean()
print "10 second as scan number : ",ten_sec2scan_avg

10 second as minite :  0.166666666667
10 second as scan number :  44.1916023521


# filter by Qual (75)

In [3]:
raw_2D_Qual75 = raw_2D[ raw_2D['Qual']> 75]

print raw_2D_Qual75.shape,raw_2D_Qual75.size

print "Reduce rate : ",float(raw_2D_Qual75.size)/raw_2D.size*100
raw_2D_Qual75.head()

(6745, 9) 60705
Reduce rate :  51.5475735575


,class,sample,date,Peak,RT,scan_num,intensity,compoud,Qual
3,Y,Y7E,2015/11/17,4,8.076,2132,483523,2,90
7,Y,Y7E,2015/11/17,8,8.406,2220,6109415,Ethyl Acetate,91
8,Y,Y7E,2015/11/17,9,8.466,2236,2264273,Furan,86
9,Y,Y7E,2015/11/17,10,8.552,2259,441593,1-Propanol,80
10,Y,Y7E,2015/11/17,11,8.672,2291,176848,Butanal,90


# Group by Compound

Compound grouping 


물질은 적어도 5

In [4]:
comp_group = raw_2D_Qual75.groupby('compoud').groups
print "Estimated Compound number : ",len(comp_group)

Estimated Compound number :  295


In [6]:
comp_group_a = {}
for i in comp_group :
    if len(comp_group[i]) > 5 :
        comp_group_a[i] = comp_group[i]

### Cluster By scan number

크로마토그래피에서 같은 물질은 같은 시간대(RT = scan number)에 등장한다. 위 전제를 통해 유사한 시간에서 용출된 물질은 하나의 물질로 정의 할 것이다.

bandwidth 는 ***10 second as scan number*** 를 기준으로 잡는다

[This](https://github.com/mattnedrich/MeanShift_py/blob/master/sample_images/ms_2d_bw_2.gif?raw=true)
[Th1is](https://github.com/john8611/MeanShift_py/blob/master/sample_images/ms_2d_bw_.8.gif?raw=true)

~~그림을 보고 싶다면 앞에 ! 를 붙여주시길

In [7]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

def cluster_scan(x,bandwidth_a) :
    # x = list , bandwidth_a = range list
    X = np.array(zip(x,np.zeros(len(x))), dtype=np.int)
    bandwidth = bandwidth_a
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(X)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    result = []
    for k in range(n_clusters_):
        my_members = labels == k
        result.append(X[my_members, 0])
    return result

In [8]:
Comp_pocket = {}
for i in comp_group_a :
    # print i 
    comp_subgroup = raw_2D.iloc[comp_group_a[i],[1,2,3,4,5,6,7,8]]
    comp_subgroup_scan = comp_subgroup['scan_num'].values.tolist()
    # print cluster_scan(comp_subgroup_scan,ten_sec2scan_avg)
    Comp_pocket[i] = cluster_scan(comp_subgroup_scan,ten_sec2scan_avg)


In [11]:
P = 0 
for i in Comp_pocket :
    # print i , len(Comp_pocket[i])
    P = P + len(Comp_pocket[i])

print "Need to search : ",P

Need to search :  513
